In [1]:
import numpy as np
import rasterio

In [2]:
sent_path = 'res/sent1-march-v1/2022-03-11.tif'
smap_path = 'res/smapL4-march-v1/2022-03-11.tif'

In [3]:
# Load the Sentinel-1 data
with rasterio.open(sent_path) as dataset:
    sentinel_data = dataset.read()

# Load the SMAP soil moisture data
with rasterio.open(smap_path) as dataset:
    soil_moisture = dataset.read()



In [4]:
# Rescale and normalize the Sentinel-1 data
sentinel_data = (sentinel_data - np.min(sentinel_data)) / (np.max(sentinel_data) - np.min(sentinel_data))

# Convert the SMAP soil moisture data to the same shape as the Sentinel-1 data
soil_moisture = np.tile(soil_moisture, (sentinel_data.shape[1], sentinel_data.shape[2], 1))
soil_moisture = soil_moisture.transpose((2, 0, 1))

MemoryError: Unable to allocate 63.0 GiB for an array with shape (121830, 138744) and data type float32

In [10]:
# Rescale and normalize the Sentinel-1 data
sentinel_data = (sentinel_data - np.min(sentinel_data)) / (np.max(sentinel_data) - np.min(sentinel_data))

# Downsample the SMAP data by a factor of 4
soil_moisture = block_reduce(soil_moisture, block_size=(1, 4, 4), func=np.mean)



In [7]:
# Rescale and normalize the Sentinel-1 data
sentinel_data = (sentinel_data - np.min(sentinel_data)) / (np.max(sentinel_data) - np.min(sentinel_data))

# Downsample the SMAP data by a factor of 4
soil_moisture = block_reduce(soil_moisture, block_size=(1, 4, 4), func=np.mean)


NameError: name 'block_reduce' is not defined

In [8]:
%pip install scikit-image


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 1.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.4/219.4 kB 2.1 MB/s eta 0:00:002.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
from skimage.measure import block_reduce

In [11]:
# Tile the SMAP data to match the Sentinel-1 data
soil_moisture = np.tile(soil_moisture, (sentinel_data.shape[1], sentinel_data.shape[2], 1))
soil_moisture = soil_moisture.transpose((2, 0, 1))

In [12]:
np.save('sentinal1.npy',sentinel_data)
np.save('smap.npy',soil_moisture)

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [5]:
# Load the preprocessed Sentinel-1 and SMAP data
sentinel_data = np.load('sentinal1.npy')
smap_data = np.load('smap.npy')

In [11]:
# smap_data = soil_moisture
smap_data.shape
# sentinel_data.shape

(93, 310, 36942)

In [6]:



# Tile the SMAP data to match the Sentinel-1 data
smap_data = np.tile(smap_data, (sentinel_data.shape[1], sentinel_data.shape[2], 1))
smap_data = smap_data.transpose((2, 0, 1))

# Flatten the data into 2D arrays
sentinel_data = sentinel_data.reshape(sentinel_data.shape[0], -1)
smap_data = smap_data.reshape(smap_data.shape[0], -1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sentinel_data, smap_data, test_size=0.3, random_state=42)

# Train a random forest regressor on the training data
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf.predict(X_test)

# Evaluate the performance of the model on the testing data
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Root mean squared error: {rmse:.2f}")
print(f"Coefficient of determination (R^2): {r2:.2f}")


MemoryError: Unable to allocate 1.20 TiB for an array with shape (310, 1065037860) and data type float32

In [ ]:
from joblib import dump


# Save the trained model to a file
dump(rf, 'random_forest_model.joblib')